In [150]:
# THIS NOTEBOOK IS RETRIEVING DETAILS ABOUT SOME BOOKS FROM `STIINTABANILOR.RO` AND LOAD THEM INTO THE PSQL DB
from bs4 import BeautifulSoup
from pprint import pprint
import requests

page = requests.get("https://stiintabanilor.ro/carti")
soup = BeautifulSoup(page.content, 'html.parser')
title = soup.title.text # gets you the text of the <title>(...)</title>

# Extract title of page
page_title = soup.title.text

# print the result
print(page_title)

Cărți - StiintaBanilor


In [151]:
# Make a request
page = requests.get(
    "https://stiintabanilor.ro/carti")
soup = BeautifulSoup(page.content, 'html.parser')

# Extract title of page
page_title = soup.title.text
print(page_title)

# Extract body of page
page_body = soup.body

# Extract head of page
page_head = soup.head

# print the result
# print(page_body, page_head)
mydivs = soup.find_all("a", class_="archive-books-grid-col")
# print(mydivs)

book_titles = [title.text for title in soup.find_all('h3', {'class' : 'book-title'})]
book_stats = [stats.text for stats in soup.find_all('div', {'class': 'book-stats-details'})]
book_stats = [ ','.join(x) for x in zip(book_stats[0::2], book_stats[1::2]) ]
book_urls = [i.get('href') for i in mydivs]


# [i.get('href') for i in mydivs]

books_dict = [(book_titles[i],book_urls[i],book_stats[i]) for i in range(len(book_titles))]
print(type(books_dict))
print(books_dict)

# for i in range(len(book_titles)):
#     print((book_titles[i],book_urls[i],book_stats[i]))


Cărți - StiintaBanilor
<class 'list'>
[('Banii. Stăpânește jocul – Tony Robbins', 'https://stiintabanilor.ro/carti/banii-stapaneste-jocul-tony-robbins/', ' 2014, 800 pag.'), ('Secretele minții de milionar – T. Harv Eker', 'https://stiintabanilor.ro/carti/secretele-mintii-de-milionar-t-harv-eker/', ' 2019, 224 pag.'), ('Drumul către libertatea financiară – Bodo Schäfer', 'https://stiintabanilor.ro/carti/drumul-catre-libertatea-financiara-bodo-schafer/', ' 2011, 384 pag.'), ('Cel mai bogat om din Babilon – George S. Clason', 'https://stiintabanilor.ro/carti/cel-mai-bogat-om-din-babilon-george-s-clason/', ' 1926, 144 pag.'), ('Copil bogat, copil isteț – Robert Kiyosaki', 'https://stiintabanilor.ro/carti/copil-bogat-copil-istet-robert-kiyosaki/', ' 2008, 296 pag.'), ('Million Dollar Habits – Brian Tracy', 'https://stiintabanilor.ro/carti/million-dollar-habits-brian-tracy/', ' 2017, 252 pag.'), ('The Psychology of Money – Morgan Housel', 'https://stiintabanilor.ro/carti/the-psychology-of-mo

In [154]:
import psycopg2

def insert_records(books_dict, connection):
  cursor = connection.cursor()
  args = ','.join(['%s'] * len(books_dict))
  sql = "insert into carti_stiinta_banilor (titlu, url, date) values {}".format(args)

  print (cursor.mogrify(sql, books_dict).decode('utf8'))

  cursor.execute(sql, books_dict)
  connection.commit()
  count = cursor.rowcount
  
  cursor.close()

  return count

def inserting(books_dict):
  try:
    connection = psycopg2.connect(user="postgres",
      password="2002",
      host="127.0.0.1",
      port="5432",
      database="web_scraping")

    count = insert_records(books_dict, connection)
    print(count, " Record(s) inserted successfully into carti_stiinta_banilor table") # SUCCESS TO THE LOG TABLE


  except (Exception, psycopg2.Error) as error:
    print("Failed to insert record into the table: \n", error) # ERR TO THE LOG TABLE

  finally:
    # closing database connection.
    if connection:
      connection.close()
      print("PostgreSQL connection is closed") # STATUS TO THE LOG TABLE

In [155]:
inserting(books_dict)

insert into carti_stiinta_banilor (titlu, url, date) values ('Banii. Stăpânește jocul – Tony Robbins', 'https://stiintabanilor.ro/carti/banii-stapaneste-jocul-tony-robbins/', ' 2014, 800 pag.'),('Secretele minții de milionar – T. Harv Eker', 'https://stiintabanilor.ro/carti/secretele-mintii-de-milionar-t-harv-eker/', ' 2019, 224 pag.'),('Drumul către libertatea financiară – Bodo Schäfer', 'https://stiintabanilor.ro/carti/drumul-catre-libertatea-financiara-bodo-schafer/', ' 2011, 384 pag.'),('Cel mai bogat om din Babilon – George S. Clason', 'https://stiintabanilor.ro/carti/cel-mai-bogat-om-din-babilon-george-s-clason/', ' 1926, 144 pag.'),('Copil bogat, copil isteț – Robert Kiyosaki', 'https://stiintabanilor.ro/carti/copil-bogat-copil-istet-robert-kiyosaki/', ' 2008, 296 pag.'),('Million Dollar Habits – Brian Tracy', 'https://stiintabanilor.ro/carti/million-dollar-habits-brian-tracy/', ' 2017, 252 pag.'),('The Psychology of Money – Morgan Housel', 'https://stiintabanilor.ro/carti/the-p